In [1]:
from fastai import *
from fastai.text import *
from fastai.core import *
import pandas as pd
import numpy as np

The barney_style.csv was prepared by webscraping the sub-title of all EP of the serie How I Met Your Mother and filtered only parts of Barney from <a href="http://transcripts.foreverdreaming.org/viewforum.php?f=177">this link</a>.
<br>Please refer to another notebook for the webscraping and data preparation.

In [2]:
df = pd.read_csv('../data/barney_style.csv')

In [3]:
df.head()

,txt
0,"(on the phone) hey, so you know how I've alway..."
1,"Okay, meet me at the bar in fifteen minutes, a..."
2,Where's your suit!? Just once when I say suit ...
3,It was a blazer!
4,I see what this is about. Have you forgotten w...


In [4]:
valid_pct = 0.15 #validation percent
df = df.iloc[np.random.permutation(len(df))]
cut = int(valid_pct * len(df)) + 1
train_df, valid_df = df[cut:], df[:cut]

In [5]:
data_lm = TextLMDataBunch.from_df('data', train_df, valid_df, text_cols='txt')

<h2>With pretrained</h2>

In [7]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,5.526078,4.896950,0.195713


In [8]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy
1,4.600121,3.941047,0.274016


In [9]:
wd=1e-7
lr=1e-3
lrs = lr

In [10]:
learn.fit(15,lrs, wd)

epoch,train_loss,valid_loss,accuracy
1,3.943179,3.554131,0.296676
2,3.778021,3.438763,0.296151
3,3.650821,3.369596,0.306381
4,3.547195,3.330213,0.313692
5,3.462643,3.292361,0.316873
6,3.389879,3.268069,0.323291
7,3.322060,3.253268,0.323741
8,3.255729,3.245859,0.323099
9,3.196164,3.239861,0.325306
10,3.136455,3.232873,0.326872


Generate example texts

In [13]:
from tqdm import trange
n = 30
speeches_w_pre = []

for i in trange(n):
    speeches_w_pre.append(learn.predict("xxbos xxfld 1", n_words=50, temperature=0.8).replace("xxbos","").replace("xxmaj","")\
.replace("xxfld","").replace("1","").replace("xxup",""))

100%|██████████| 30/30 [01:51<00:00,  3.69s/it]


In [21]:
pretrained_df = pd.DataFrame({'txt':speeches_w_pre})
pretrained_df.to_csv('pretrained.csv',index=False)

<h2>Without pretrained</h2>

In [12]:
learn = language_model_learner(data_lm, bptt = 70, emb_sz = 400, nh = 1150, nl = 3,
                                  drop_mult = 0.5, bias = False, qrnn = False, 
                                  alpha=2, beta = 1, 
                                  pretrained_fnames = None)

In [13]:
learn.metrics = [accuracy]
learn.opt_func = partial(optim.Adam, betas=(0.8, 0.99))

lr = 1e-3
wd = 1e-7
learn.wd=wd

learn.fit_one_cycle(cyc_len = 15, 
                    max_lr= lr,
                    div_factor= 20,
                    moms = (0.8, 0.7),
                    pct_start = 0.1,
                    wd = wd
                   ) 


epoch,train_loss,valid_loss,accuracy
1,6.547472,5.116069,0.118082
2,5.658830,4.897634,0.117916
3,5.276206,4.520227,0.182300
4,4.993814,4.401872,0.214932
5,4.780506,4.253617,0.226896
6,4.610217,4.152420,0.227628
7,4.471513,4.068677,0.240415
8,4.344780,3.962299,0.255792
9,4.240228,3.908992,0.260157
10,4.156411,3.878998,0.260840


In [14]:
n = 30
speeches_wo_pre = []

for i in trange(n):
    speeches_wo_pre.append(learn.predict("xxbos xxfld 1", n_words=50, temperature=0.8).replace("xxbos","").replace("xxmaj","")\
.replace("xxfld","").replace("1","").replace("xxup",""))

100%|██████████| 30/30 [01:50<00:00,  3.70s/it]


In [16]:
wo_pretrained_df = pd.DataFrame({'txt':speeches_wo_pre})
wo_pretrained_df.to_csv('wo_pretrained.csv',index=False)

In [17]:
# randomly pick 3 sentences from each
#pretrained_df = pd.read_csv('pretrained.csv')

In [24]:
print('=====real text=====\n')
for txt in np.random.choice(df['txt'].values,3): print(txt+'\n')
print('=====with pretrained=====\n')
for txt in np.random.choice(pretrained_df['txt'].values,3): print(txt+'\n')
print('=====without pretrained=====\n')
for txt in np.random.choice(wo_pretrained_df['txt'].values,3):print(txt+'\n')

=====real text=====

And I only stirred that with my pinky.

Oh, Marshall, you only think you're happy. Just wait, because tonight, (imitating Oprah) is Barney's Favorite Things!

This place in the Catskills. Look it up. I was there teaching for the summer.

=====with pretrained=====

   , it 's the road ?  just for you ,  wendy .   ted , i 'm ...   thanks .   it was not the end of a  black  this is the worst .   that 's it . i 'm

   you 're gon na try to help people out there .  that 's when they 're talking about something ... (  ted picks up a girl and then laughing )  anyway , you 'll never forget why we 're not gon na play a game well .

    ok , that 's the way it takes to settle down .   we 're doing this .   thank you ,  marshall .   ok !  marshall , i do n't want to see it !   oh , my  god

=====without pretrained=====

   .  okay , this 's 's a women .   plus , i they got the chinese .  he 're totally of in that .  hair them .   oh , here .  what ...  ted , i 'm ?   but

   